In [32]:
import pandas as pd
from modules.UCKMeans import UCKMeans as UC
from modules.UCOSKMeans import UCOSKMeans as UCOS
from modules.SortMeans import SortMeans as SM
from modules.Household import Household

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
    
# Mongo 연결
from pymongo import MongoClient as mc
import random
import pprint as pp

#  print(uid_check)
# custom module
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

SEASONS = {
    "봄": [3, 4, 5],
    "여름": [6, 7, 8],
    "가을": [9, 10, 11],
    "겨울": [12, 1, 2]
}

SEASONSARR = ['봄', '여름', '가을', '겨울']

DAYARR = ["월", "화", "수", "목", "금", "토", "일"]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
mongo_uri = "mongodb://localhost:27017"
client = mc(mongo_uri)

keti_pr_db = client.keti_pattern_recognition
household_col = keti_pr_db.household_info

db_datas = household_col.find()
og_datas = []

for idx, data in enumerate(db_datas):
    if (idx) == 30:
        break
        
    if (((idx) / 1) % 5) == 0:
        print("{}: {}".format(idx, data['uid']))
    
    uid, timeslot = data['uid'], data['timeslot']
    
    datelist = [
        dt.strptime(ts['time'], "%Y-%m-%d T%H:%M %z").date()
        for ts in timeslot
    ]
    datelist = list(set(datelist))
    datelist.sort()
    
    ts_datas = {}
    start_idx = 0
    end_idx = 96
    enl = 1

    for date in datelist:
        ts_datas[date] = [ ts['power'] * enl for ts in timeslot[start_idx:end_idx]]
        start_idx = end_idx
        end_idx = end_idx + 96
    
    ts_datas = pd.DataFrame(ts_datas).T
    datas = ts_datas.reset_index().copy()

    datas.rename(columns={"index": "date"}, inplace=True)
    datas['date'] = pd.to_datetime(datas['date'])
    datas['month'] = [dt.month for dt in datas['date']]
    datas = [
        datas[(datas['month'].isin(SEASONS[season]))].copy()
        for season in SEASONSARR
    ]

    bak_datas = datas.copy()

    season_datas = {
        "봄": datas[0].copy(),
        "여름": datas[1].copy(),
        "가을": datas[2].copy(),
        "겨울": datas[3].copy()
    }
    
    household = Household(uid, season_datas)
    household.merging()
    og_datas.append(household)
    
client.close()

0: 아파트1-104-1206
5: 아파트1-104-203
10: 아파트1-103-502
15: 아파트1-102-903
20: 아파트1-102-403
25: 아파트1-101-1202


In [87]:
TSS_LIST = pd.DataFrame(columns=['tss'])

for data in og_datas:
    TSS_LIST.loc[data.uid] = [
        data.TSS()
    ]

# HouseHold K
uid_check = []
target_datas = {}
season = "봄"
h_k = 6
import random

while True:
    ran_idx = random.randrange(0, 30)
    uid_tmp = og_datas[ran_idx].uid
    if uid_tmp not in uid_check:
        uid_check.append(uid_tmp)
        target_datas[uid_tmp] = og_datas[ran_idx].merge_datas[season]
    else:
        continue
    
    if len(uid_check) == 6:
        break

# Map
'''
    uid {
        date [
        
        ]
        date {
        
        }
    }
'''
# target_datas[uid][date].values
all_datas = pd.DataFrame(columns=[idx for idx in range(0,24)])
all_datas
tmp
for uid in uid_check:
    tmp = pd.DataFrame(columns=[idx for idx in range(0,24)])
    tmp.index.names = "uid": ["date"]]
    for date in target_datas[uid]:
        tmp
#         tmp.loc[(uid, date)] = target_datas[uid][date].values
# all_datas.set_index(['uid', 'date'], inplace=True)
# all_datas_col = all_datas.T
# target_datas

KeyError: 0

In [6]:
all_datas_col

uid      아파트1-101-1602                                                         \
date        2018-05-01 2018-05-01 2018-05-01 2018-05-01 2018-05-01 2018-05-01   
timeslot         0.000      1.000      2.000      3.000      4.000      5.000   
data             0.431      0.178      0.205      0.198      0.204      0.251   

uid                                                   ... 아파트1-101-1604  \
date     2018-05-01 2018-05-01 2018-05-01 2018-05-01  ...    2019-04-30   
timeslot      6.000      7.000      8.000      9.000  ...        14.000   
data          0.231      0.234      0.228      0.328  ...         0.056   

uid                                                                         \
date     2019-04-30 2019-04-30 2019-04-30 2019-04-30 2019-04-30 2019-04-30   
timeslot     15.000     16.000     17.000     18.000     19.000     20.000   
data          0.053      0.047      0.046      0.105      0.269      0.355   

uid                                        
date     2019-04-30 2019-04-30 2019-04-30  
timeslot     21.000     22.000      23.00  
data          0.154      0.153       0.15  

[2 rows x 13248 columns]

In [10]:
from modules.SeasonMeans import SeasonMeans as SM

sm = SM(og_datas, all_datas)
sm.dimension_reduction()

---Season Means---
---Dimension Reduction---


{'봄':     2018-05-01  2018-05-02  2018-05-03  2018-05-04  2018-05-05  2018-05-06  \
 0        0.341       0.275       0.183       0.309       0.305       0.397   
 1        0.337       0.201       0.235       0.308       0.179       0.409   
 2        0.324       0.176       0.167       0.309       0.180       0.400   
 3        0.319       0.210       0.165       0.309       0.172       0.384   
 4        0.235       0.199       0.163       0.311       0.171       0.276   
 5        0.169       0.202       0.164       0.217       0.174       0.206   
 6        0.200       0.199       0.162       0.218       0.169       0.200   
 7        0.171       0.252       0.173       0.212       0.164       0.199   
 8        0.170       0.213       0.226       0.242       0.162       0.199   
 9        0.172       0.173       0.178       0.187       0.166       0.200   
 10       0.199       0.186       0.167       0.211       0.169       0.198   
 11       0.169       0.188       0.212       0